In [1]:
import awkward as ak
import numpy as np
import numba as nb
import hist
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep as hep
from common import load_events, DelphesSchema

In [2]:
class DelphesSchema2(DelphesSchema):
    def __init__(self, base_form):
        for key in list(base_form["contents"].keys()):
            if "fBits" in key:
                base_form["contents"].pop(key, None)
        super().__init__(base_form)

In [3]:
# load data
events = load_events(
    "models/s-channel_mmed-1000_Nc-2_Nf-2_scale-35.1539_mq-10_mpi-20_mrho-20_pvector-0.75_spectrum-cms_rinv-0.3/events.root",
    schema=DelphesSchema2,
)

In [152]:
from numpy.typing import NDArray
from typing import List

@nb.njit("i8[:](i8[:],u4[:])")
def get_constituents_kernel(jet_refs: NDArray[np.int64], cand_ids: NDArray[np.uint32]) -> NDArray[np.int64]:
#@nb.njit
#def get_constituents_kernel(jet_refs, cand_ids):
    # get hash table mapping global index : global unique ID
    hash_table = {k:v for v,k in enumerate(cand_ids)}

    # apply hash map
    output = [hash_table[ref] for ref in jet_refs]

    return np.asarray(output)

def get_constituents(events_in, jetsname, candsname, nevents=-1):
    events = events_in if nevents<0 else events_in[0:nevents]
    output = []
    ctr = 0
    for jets,cands in zip(events[jetsname], events[candsname]):
        #print(ctr)
        #ctr+=1
        indices = get_constituents_kernel(ak.flatten(jets.Constituents.refs).to_numpy(), cands.fUniqueID.to_numpy()) if jets is not None else []
        if len(indices)==0:
            unflattened = None
        else:
            unflattened = ak.unflatten(cands[indices],ak.count(jets.Constituents.refs,axis=1),behavior=cands.behavior)[None]
        output.append(unflattened)
    return ak.with_name(ak.concatenate(output), "Particle")

In [5]:
ak.type(events["ParticleFlowCandidate","fUniqueID"])

1000 * var * uint32[parameters={"__doc__": "fUniqueID[ParticleFlowCandidate_]"}]

In [6]:
ak.type(events["FatJet","Constituents","refs"])

1000 * var * var * int64

In [18]:
print(ak.count(events[0,"FatJet"].Constituents.refs,axis=1).to_numpy())

[54 11  8]


In [153]:
%%timeit -n 1 -r 5
#get_constituents(events,"FatJet","ParticleFlowCandidate",1000)
events["FatJet","ConstituentsFull"] = get_constituents(events,"FatJet","ParticleFlowCandidate")

2.76 s ± 27.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [21]:
def sum_4vec(vec):
    summed_vec = {
        "t": ak.sum(vec.E,axis=-1),
        "x": ak.sum(vec.px,axis=-1),
        "y": ak.sum(vec.py,axis=-1),
        "z": ak.sum(vec.pz,axis=-1),
    }
    return ak.zip(summed_vec,with_name="LorentzVector")
check_jets = sum_4vec(events.FatJet.ConstituentsFull)
print(check_jets.mass-events.FatJet.mass)

[[0.000565, 1.43e-06, -3.81e-06], [1.91e-05, ... [0.000404, 6.1e-05, 0.000409]]


In [25]:
get_constituents_kernel.signatures

[]

In [36]:
type(list(np.ndenumerate(events[0]["ParticleFlowCandidate"].fUniqueID.to_numpy()))[0][0][0])

int

In [48]:
type(np.array([1,2,3])[0])

numpy.int64

In [110]:
from numpy.typing import NDArray

def vec_translate(a, d):
    return np.vectorize(d.__getitem__)(a)

def get_constituents_perevent(jet_refs: NDArray[np.int64], cand_ids: NDArray[np.uint32]) -> NDArray[np.int64]:
    # get hash table mapping global index : global unique ID
    hash_table = {k:v for v,k in enumerate(cand_ids)}

    # apply hash map
    translated = vec_translate(jet_refs,hash_table)

    return np.asarray(translated)

def get_constituents_new(events_in, jetsname, candsname, nevents=-1):
    events = events_in if nevents<0 else events_in[0:nevents]
    output = []
    ctr = 0
    for jets,cands in zip(events[jetsname], events[candsname]):
        #print(ctr)
        #ctr+=1
        indices = get_constituents_perevent(ak.flatten(jets.Constituents.refs).to_numpy(), cands.fUniqueID.to_numpy()) if jets is not None else []
        if len(indices)==0:
            unflattened = None
        else:
            unflattened = ak.unflatten(cands[indices],ak.count(jets.Constituents.refs,axis=1),behavior=cands.behavior)[None]
        output.append(unflattened)
    return ak.with_name(ak.concatenate(output), "Particle")

In [114]:
%%timeit -n 1 -r 5
get_constituents_new(events,"FatJet","ParticleFlowCandidate",1000)

2.89 s ± 36.1 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [76]:
test_fmap = [(0,2003),(1,2001),(2,2005)]
sorted_test_fmap = np.sort(test_fmap,axis=-1)
print(sorted_test_fmap)
search_idx = np.searchsorted(sorted_test_fmap[1,:],sorted([2005,2003,2001]),side='left')
print(search_idx)

[[   0 2003]
 [   1 2001]
 [   2 2005]]
[1 2 2]


In [79]:
test_ids = np.array([2003,2001,2005])
asort_ids = np.argsort(test_ids)
ind_ids = np.arange(0, len(test_ids))
test_ids = test_ids[asort_ids]
ind_ids = ind_ids[asort_ids]
print(test_ids)
search_idx = np.searchsorted(test_ids,[2005,2003,2001])
print(search_idx)
print(ind_ids[search_idx])

[2001 2003 2005]
[2 1 0]
[2 0 1]


In [139]:
from numpy.typing import NDArray

def get_constituents_sorted(jet_refs: NDArray[np.int64], cand_ids: NDArray[np.uint32]) -> NDArray[np.int64]:
    # get hash table mapping global index : global unique ID
    asort = np.argsort(cand_ids)
    cand_ids = cand_ids[asort]
    cand_inds = np.arange(0, len(cand_ids))[asort]
    search_idx = np.searchsorted(cand_ids, jet_refs)
    ref_idx = cand_inds[search_idx]
    return ref_idx

def get_constituents_new2(events_in, jetsname, candsname, nevents=-1):
    events = events_in if nevents<0 else events_in[0:nevents]
    output = []
    ctr = 0
    for jets,cands in zip(events[jetsname], events[candsname]):
        #print(ctr)
        #ctr+=1
        indices = get_constituents_sorted(ak.flatten(jets.Constituents.refs).to_numpy(), cands.fUniqueID.to_numpy()) if jets is not None else []
        if len(indices)==0:
            unflattened = None
        else:
            unflattened = ak.unflatten(cands[indices],ak.count(jets.Constituents.refs,axis=1),behavior=cands.behavior)[None]
        output.append(unflattened)
    return ak.with_name(ak.concatenate(output), "Particle")

In [134]:
%%timeit -n 1 -r 5
get_constituents_new2(events,"FatJet","ParticleFlowCandidate",1000)

1.95 s ± 11.2 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [121]:
test_arr = np.full(10,-1)
test_inds = np.array([0,4,7])
test_vals = np.array([1,2,3])
test_arr[test_inds] = test_vals
print(test_arr)

[ 1 -1 -1 -1  2 -1 -1  3 -1 -1]


In [127]:
from numpy.typing import NDArray

def get_constituents_grow(jet_refs: NDArray[np.int64], cand_ids: NDArray[np.uint32]) -> NDArray[np.int64]:
    # shift to start at zero
    min_id = np.min(cand_ids)
    cand_ids = cand_ids - min_id
    cand_inds = np.arange(0, len(cand_ids))
    cand_dense = np.full(np.max(cand_ids)+1, -1)
    cand_dense[cand_ids] = cand_inds
    jet_refs = jet_refs - min_id
    ref_idx = cand_dense[jet_refs]
    return ref_idx

def get_constituents_new3(events_in, jetsname, candsname, nevents=-1):
    events = events_in if nevents<0 else events_in[0:nevents]
    output = []
    ctr = 0
    for jets,cands in zip(events[jetsname], events[candsname]):
        #print(ctr)
        #ctr+=1
        indices = get_constituents_grow(ak.flatten(jets.Constituents.refs).to_numpy(), cands.fUniqueID.to_numpy()) if jets is not None else []
        if len(indices)==0:
            unflattened = None
        else:
            unflattened = ak.unflatten(cands[indices],ak.count(jets.Constituents.refs,axis=1),behavior=cands.behavior)[None]
        output.append(unflattened)
    return ak.with_name(ak.concatenate(output), "Particle")

In [131]:
%%timeit -n 1 -r 5
get_constituents_new3(events,"FatJet","ParticleFlowCandidate",1000)

2.9 s ± 65.8 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [145]:
test_arr2 = np.array([1,2,3,4])
print(test_arr2[0:])

[1 2 3 4]


In [158]:
ak.max(ak.count(events["ParticleFlowCandidate"].fUniqueID,axis=1))

374

In [173]:
import timeit

def time_get_constituents(events, jetsname, candsname, getter):
    global jets
    global cands
    all_times = []
    for jets,cands in zip(events[jetsname], events[candsname]):
        if jets is not None and cands is not None:
            times = timeit.repeat("{}(ak.flatten(jets.Constituents.refs).to_numpy(), cands.fUniqueID.to_numpy())".format(getter), number=1, globals=globals())
            print(len(ak.flatten(jets.Constituents.refs)),len(cands),np.mean(times))
            all_times.extend(times)
    print(np.mean(all_times))

In [174]:
time_get_constituents(events,"FatJet","ParticleFlowCandidate","get_constituents_kernel")

73 172 0.00139721455052495
65 141 0.0010531993582844734
101 246 0.0007189780008047819
70 158 0.0006530474871397019
66 136 0.0016435482539236546
80 136 0.001364251784980297
97 167 0.0010517633985728026
60 172 0.0007047003600746393
59 202 0.0006533495616167784
61 189 0.000522550567984581
99 270 0.0004886365961283446
36 104 0.00047789020463824273
101 171 0.0004984824452549219
66 81 0.0004735852126032114
161 310 0.0005064035765826702
86 109 0.0004734703805297613
110 281 0.00048059760592877865
107 193 0.0004813028033822775
100 203 0.00047619263641536237
146 248 0.00047915917821228504
50 131 0.0004680641461163759
130 181 0.0004740973934531212
119 263 0.0005390146747231483
99 178 0.0005017154384404421
100 226 0.0005016377661377191
63 158 0.0005028369370847941
94 195 0.0005006926134228706
65 189 0.0004993719048798084
27 97 0.00049037323333323
48 101 0.0004884530324488878
101 176 0.0004900151863694191
18 90 0.00047364379279315474
46 87 0.0004607412498444319
45 189 0.0004695332143455744
88 191 0

57 166 0.0004733963869512081
48 115 0.0004775704350322485
73 97 0.0004608205985277891
164 278 0.0004868177697062492
101 192 0.00046803317964076995
90 205 0.0004959381651133299
85 261 0.00046206405386328697
75 133 0.0004570942372083664
45 136 0.00045988047495484354
46 71 0.00045386506244540217
78 107 0.00045580174773931505
59 129 0.0004546763375401497
101 195 0.0004603547975420952
51 83 0.00045205121859908103
25 124 0.0004544387571513653
65 98 0.0004543124698102474
86 164 0.00045804176479578016
68 152 0.000456136791035533
50 169 0.0004620513878762722
130 194 0.00045969197526574135
135 294 0.00046551874838769437
99 229 0.00046122763305902483
61 128 0.0004573849495500326
29 58 0.0004517088178545237
94 214 0.00046798135153949263
72 205 0.00046127159148454664
137 272 0.0004653074312955141
92 165 0.00045831198804080486
79 183 0.00046517415903508664
49 121 0.0004588368348777294
141 213 0.00046799038536846636
167 280 0.0004701063502579927
66 215 0.00047395662404596804
130 304 0.000469272350892

In [175]:
time_get_constituents(events,"FatJet","ParticleFlowCandidate","get_constituents_sorted")

73 172 0.0015885577537119388
65 141 0.00105810877867043
101 246 0.0008762868586927652
70 158 0.0006421510130167008
66 136 0.0007725129835307598
80 136 0.0017989686224609614
97 167 0.0016313774045556783
60 172 0.0009818269871175289
59 202 0.0008938435930758715
61 189 0.0011594831477850676
99 270 0.0017327409703284502
36 104 0.001080869184806943
101 171 0.0008521121460944414
66 81 0.0007316927891224623
161 310 0.0007057749666273594
86 109 0.0006187769584357738
110 281 0.0006326160393655301
107 193 0.000624156603589654
100 203 0.0006166803650557995
146 248 0.0006218431983143092
50 131 0.0005441037937998772
130 181 0.0006526215467602015
119 263 0.0006147359497845172
99 178 0.00061191082932055
100 226 0.0006064347457140684
63 158 0.0006148960441350937
94 195 0.0006004511844366789
65 189 0.0005445064045488835
27 97 0.0005139863584190607
48 101 0.000491135474294424
101 176 0.0006792835891246795
18 90 0.0004958038218319416
46 87 0.0005426861811429262
45 189 0.000696220574900508
88 191 0.000599

121 271 0.0007231052033603192
96 132 0.0005398999433964491
134 255 0.0007061139680445194
85 99 0.0006040795706212521
95 216 0.0005648022517561913
87 189 0.000531590823084116
130 266 0.0006563665810972452
89 229 0.0015219039749354124
89 190 0.0015697654336690903
101 214 0.0010703308042138814
116 193 0.0008465209975838661
71 162 0.0007176482118666172
27 138 0.00048754201270639896
84 274 0.000668067205697298
35 73 0.0004920975770801306
28 120 0.0005332432221621275
70 189 0.000719003425911069
50 179 0.0005446220748126507
75 159 0.0006478277500718832
74 185 0.0005660239607095718
91 163 0.0005946517921984196
131 241 0.0006029052194207907
131 181 0.0005956187844276429
52 98 0.0004910519812256098
90 231 0.000650243042036891
59 130 0.0005949229933321476
64 134 0.0005957542452961206
154 250 0.0006015156395733356
100 133 0.0005967773497104645
136 249 0.000611440883949399
62 83 0.0005369107238948345
59 160 0.0006328439339995384
53 133 0.0005008523818105459
8 44 0.0005238571669906378
115 235 0.0006

109 282 0.001810837211087346
103 213 0.001050355052575469
53 129 0.0010309533681720495
51 147 0.0017311539500951766
71 154 0.0019351411610841751
78 163 0.0011865793727338313
35 122 0.0008867633063346147
93 143 0.0007802580017596483
37 156 0.0007318450137972832
124 291 0.0006540450267493725
69 145 0.0006771134678274394
95 155 0.0006074175238609314
85 154 0.0006107278168201447
97 176 0.0005469214636832475
58 171 0.000542947230860591
139 150 0.0006459166295826435
57 83 0.0006060411687940359
75 226 0.000547907454892993
44 165 0.0006746452301740647
49 100 0.0004995403811335564
72 208 0.0006518945563584566
55 181 0.0006179743912070989
86 127 0.0005026194732636214
92 144 0.0004907328635454178
124 220 0.0007604508195072412
111 194 0.000603005476295948
40 104 0.000490607600659132
82 172 0.0016501569654792548
92 127 0.0010595953557640315
98 192 0.0011556898709386588
149 269 0.0010362709406763315
145 335 0.0012742947787046433
122 144 0.0008224734105169773
87 224 0.0009068784303963185
101 207 0.00

75 159 0.00048811854794621467
160 333 0.0006685841362923384
74 145 0.0006763271987438202
98 172 0.0005305894184857607
161 307 0.0005354943685233593
71 180 0.0005943386349827051
120 236 0.0006058455910533667
69 168 0.0005994737148284912
102 230 0.0006053488235920667
19 102 0.0005690495483577252
101 178 0.0005969934165477753
23 57 0.0004961429629474878
135 277 0.0006622662302106618
70 189 0.0006091640330851078
51 186 0.0006014775950461626
96 205 0.000581818213686347
99 203 0.0006231058388948441
12 50 0.000612654909491539
71 207 0.000620414549484849
54 170 0.0006195770110934973
66 112 0.0005459290929138661
27 154 0.0004932250827550888
96 202 0.0005602291785180569
35 106 0.0006111097987741232
144 238 0.0005116076208651066
110 177 0.0005331729538738727
35 174 0.0016595213674008846
110 295 0.0015889849979430436
113 220 0.0011441029608249664
112 200 0.000790052069351077
54 161 0.0007939007598906755
108 150 0.0017152264714241027
56 98 0.0012303299736231565
88 245 0.001130469050258398
67 179 0.

In [176]:
time_get_constituents(events,"FatJet","ParticleFlowCandidate","get_constituents_perevent")

73 172 0.0019632982090115546
65 141 0.001240009581670165
101 246 0.0009453914128243924
70 158 0.0007755838334560395
66 136 0.0006139660254120827
80 136 0.0006369850598275661
97 167 0.0006826771423220634
60 172 0.0006257012486457824
59 202 0.000621166592463851
61 189 0.0006048487965017557
99 270 0.0006474645808339119
36 104 0.0005628653336316348
101 171 0.0006302705500274896
66 81 0.000583338551223278
161 310 0.0007359863724559545
86 109 0.0006017947569489479
110 281 0.0006462967023253441
107 193 0.0006515759974718094
100 203 0.0006220428738743066
146 248 0.0007264907937496901
50 131 0.0005762803368270398
130 181 0.0006792109925299883
119 263 0.0006519286427646876
99 178 0.000612888764590025
100 226 0.0006143743637949228
63 158 0.0005695893429219723
94 195 0.0006206244230270386
65 189 0.0005789903458207846
27 97 0.0005330154206603765
48 101 0.0005715410225093365
101 176 0.0006049607880413532
18 90 0.0005137779749929905
46 87 0.0005414814688265324
45 189 0.0005674799904227257
88 191 0.00

183 330 0.0006914372555911541
94 146 0.0006194284185767173
93 241 0.0006170714274048805
56 86 0.000554900337010622
76 169 0.0005828696768730879
100 153 0.0005958613939583302
104 186 0.0006210360210388899
96 182 0.0006011243909597397
74 188 0.0005720090121030807
106 168 0.0006028525996953249
81 142 0.0005769941955804825
67 107 0.0005605445709079504
112 245 0.0006176119670271874
126 164 0.0006232041865587235
32 130 0.0005261832382529974
88 165 0.0005874636117368937
64 114 0.0005733147263526917
105 205 0.0006215751636773348
118 267 0.0006386115681380033
92 139 0.000606590649113059
138 226 0.0006768772378563881
49 195 0.0005692565813660621
87 135 0.0005835349671542645
50 191 0.0005497904494404793
104 182 0.0006027624942362309
62 153 0.0005579274613410235
68 124 0.0005589413922280073
82 199 0.0005823007319122553
121 159 0.0006134378258138895
111 285 0.0006176705937832594
84 100 0.0005780632141977549
55 166 0.0005586313549429178
35 94 0.0005219991784542799
61 204 0.0005581016652286052
55 166

76 132 0.0010212888941168786
70 144 0.0006664623972028494
47 158 0.0006394512020051479
78 124 0.0006031493656337261
69 167 0.0005968278273940086
102 253 0.0006199527531862258
64 146 0.0005638496484607458
62 168 0.000582820875570178
97 246 0.0006018887273967266
105 188 0.0006034554913640022
85 104 0.0005786215886473655
51 57 0.0005391217768192291
61 202 0.0005627191625535488
120 306 0.0006283983588218689
59 76 0.0005474168341606855
65 131 0.0005576754454523325
49 172 0.0005536878481507302
46 189 0.0005534044001251459
70 218 0.0005718198139220476
46 64 0.0005337563809007407
113 290 0.0006200002040714026
127 152 0.0006212945561856031
111 244 0.0006114437710493803
55 163 0.0005496191326528788
47 73 0.0005321252159774303
28 85 0.0005161135923117399
78 171 0.0005783060099929571
131 215 0.0006667669396847487
167 260 0.0007101583760231733
123 231 0.0006547776516526938
158 307 0.0006721319630742073
108 136 0.000606152368709445
57 114 0.0005486827343702316
104 157 0.000599527033045888
116 149 0.

In [177]:
time_get_constituents(events,"FatJet","ParticleFlowCandidate","get_constituents_grow")

73 172 0.001816241629421711
65 141 0.001163474377244711
101 246 0.0009270311333239078
70 158 0.0008729108143597841
66 136 0.0006762112025171518
80 136 0.0006620051804929972
97 167 0.0006807614117860794
60 172 0.0007001511752605438
59 202 0.0006794952321797609
61 189 0.0006763902027159929
99 270 0.0006666768342256546
36 104 0.000630665197968483
101 171 0.0006550203077495098
66 81 0.0006283012218773365
161 310 0.0006505088414996863
86 109 0.0006270152516663074
110 281 0.0006589299999177455
107 193 0.00063465042039752
100 203 0.0006383263971656561
146 248 0.0006302872207015753
50 131 0.0006637095473706722
130 181 0.0006189610343426466
119 263 0.0006099359598010778
99 178 0.0005873357411473989
100 226 0.0005631542298942804
63 158 0.0006162060424685478
94 195 0.0006107766181230545
65 189 0.0006354033946990966
27 97 0.0018932857550680638
48 101 0.0012242077384144068
101 176 0.0008264124393463135
18 90 0.001128139579668641
46 87 0.0012636124156415462
45 189 0.0009657101705670357
88 191 0.0006

101 214 0.0005786123685538769
116 193 0.0005800122395157814
71 162 0.0005811111070215702
27 138 0.0005857238546013832
84 274 0.0006225067656487227
35 73 0.0005889072082936764
28 120 0.0005788580514490604
70 189 0.0006239806301891804
50 179 0.0006844672374427318
75 159 0.0005805664695799351
74 185 0.0005798992235213519
91 163 0.0005787400528788567
131 241 0.0006198068615049124
131 181 0.0005810463801026345
52 98 0.0005772322416305542
90 231 0.0005905502010136842
59 130 0.0005821104161441327
64 134 0.0005766445770859719
154 250 0.0007746180053800344
100 133 0.0005711090285331011
136 249 0.0005520591977983714
62 83 0.0005495761986821889
59 160 0.0005517948418855667
53 133 0.0005517933983355761
8 44 0.0005483496002852917
115 235 0.001081051817163825
56 104 0.0005509837530553341
67 229 0.0005802887957543134
152 290 0.0005788980051875114
104 203 0.0008001734036952257
127 196 0.0005770979449152946
151 225 0.0007507131900638342
79 210 0.0006904241628944874
79 221 0.0006578461732715368
190 326 

55 181 0.0007577384356409311
86 127 0.0005710325669497252
92 144 0.0005681225564330816
124 220 0.0005785782355815172
111 194 0.0005676422733813524
40 104 0.0005711583886295557
82 172 0.000757569633424282
92 127 0.0005741988774389028
98 192 0.000608627824112773
149 269 0.0006190715357661247
145 335 0.0006009253207594156
122 144 0.0005651977844536304
87 224 0.0005679028108716011
101 207 0.0006502090021967888
118 190 0.0005985474213957787
68 100 0.0005638453643769026
28 89 0.0005654235836118459
72 195 0.000654310081154108
67 281 0.0006087054032832384
104 131 0.0005920271854847669
136 287 0.000665029650554061
48 141 0.0006476381327956915
96 170 0.0006244188174605369
86 164 0.0005968842189759016
102 252 0.0005691084079444408
68 203 0.0005834242329001427
113 248 0.0005664195865392685
80 140 0.0005814884789288044
36 152 0.0006818007677793503
148 275 0.000586121017113328
95 252 0.0006245388183742761
167 205 0.000610648188740015
101 239 0.000610164925456047
47 77 0.0005640951916575432
51 145 0.

144 238 0.0007025872357189656
110 177 0.0013882745523005724
35 174 0.0015662130434066057
110 295 0.001101272413507104
113 220 0.001226669503375888
112 200 0.0013375915121287107
54 161 0.001588809583336115
108 150 0.0017576402053236962
56 98 0.0014491184148937464
88 245 0.0015186891425400973
67 179 0.001304800622165203
147 287 0.0009100665803998709
84 209 0.0008164798375219107
65 154 0.0006352773867547512
31 70 0.0005916377995163202
66 123 0.0005919193848967552
35 148 0.0005919704679399728
88 153 0.000592068349942565
80 152 0.0005921028088778258
91 236 0.0005948269739747047
45 165 0.001170729985460639
63 94 0.000590478628873825
158 262 0.0007438832428306342
51 108 0.0005901299882680178
184 358 0.0007670059334486723
126 271 0.0005919900257140398
106 236 0.0005899779498577118
93 181 0.0005919522140175104
59 109 0.0005882943980395794
69 175 0.0005924369674175977
109 174 0.0005925983656197787
56 195 0.0005907144397497178
83 237 0.0005906148813664913
32 108 0.0005895969923585653
81 101 0.000